In [17]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [20]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [21]:
train_dir = "/content/drive/MyDrive/data/train"
test_dir  = "/content/drive/MyDrive/data/test"

train_dataset = ImageFolder(train_dir, transform=transform)
test_dataset  = ImageFolder(test_dir,  transform=transform)


In [22]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [23]:
train_dataset.class_to_idx


{'chihuahua': 0, 'muffin': 1}

In [24]:
model = resnet18(weights="IMAGENET1K_V1")

# Replace final layer (1000 → 2)
model.fc = nn.Linear(512, 2)

model = model.to(device)


In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)


In [26]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}")


Epoch 1/5, Loss: 12.2985
Epoch 2/5, Loss: 7.6657
Epoch 3/5, Loss: 5.0507
Epoch 4/5, Loss: 4.1071
Epoch 5/5, Loss: 3.7574


In [27]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print("Test Accuracy:", correct / total)


Test Accuracy: 0.9493243243243243


In [28]:
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["chihuahua", "muffin"]))


Confusion Matrix:
 [[582  58]
 [  2 542]]

Classification Report:
              precision    recall  f1-score   support

   chihuahua       1.00      0.91      0.95       640
      muffin       0.90      1.00      0.95       544

    accuracy                           0.95      1184
   macro avg       0.95      0.95      0.95      1184
weighted avg       0.95      0.95      0.95      1184

